<a href="https://colab.research.google.com/github/Rivianee/data-science/blob/master/RANDOMFOREST_CATEGORIAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

# Carregar a planilha com os dados
data = pd.read_excel('/content/PRODUTO E CATEGORIA PARA TESTE.xlsx')

# Padronizar as colunas "Produto" e "Categoria"
data['Produto'] = data['Produto'].str.lower()

# Separar os dados com categorias preenchidas (treinamento)
train_data = data[data['Categoria'].notnull()]

# Separar as variáveis de treinamento e alvo
X_train_text = train_data['Produto'].values
y_train = train_data['Categoria'].values

# Criar uma instância do vetorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Transformar o texto em vetores TF-IDF
X_train = tfidf_vectorizer.fit_transform(X_train_text)

# Criar e treinar o modelo de Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

# Filtrar os itens a serem classificados (sem categoria)
unclassified_data = data[data['Categoria'].isnull()]

if len(unclassified_data) > 0:
    # Filtrar itens sem categoria
    unclassified_data = unclassified_data.dropna(subset=['Produto'])

    if len(unclassified_data) > 0:
        # Transformar o texto dos itens sem categoria em vetores TF-IDF
        X_unclassified_text = unclassified_data['Produto'].values
        X_unclassified = tfidf_vectorizer.transform(X_unclassified_text)

        # Fazer previsões para os itens sem categoria
        predicted_categories = rf_model.predict(X_unclassified)

        # Atualizar as categorias dos itens classificados
        data.loc[unclassified_data.index, 'Categoria'] = predicted_categories

        # Salvar o DataFrame atualizado em um novo arquivo Excel
        data.to_excel('seu_arquivo_atualizado.xlsx', index=False)
    else:
        print("Não há itens sem categoria para serem classificados.")
else:
    print("Não há itens para serem classificados.")

In [47]:
from sklearn.metrics import accuracy_score

# Filtrar os dados com categorias preenchidas (teste)
test_data = data[data['Categoria'].notnull()]

# Separar as variáveis de teste
X_test_text = test_data['Produto'].values
y_true = test_data['Categoria'].values

# Transformar o texto dos itens de teste em vetores TF-IDF
X_test = tfidf_vectorizer.transform(X_test_text)

# Fazer previsões nos dados de teste
y_pred = rf_model.predict(X_test)

# Calcular a acurácia do modelo
accuracy = accuracy_score(y_true, y_pred)
print(f'Acurácia do modelo: {accuracy:.2f}')

Acurácia do modelo: 1.00


In [48]:
from sklearn.model_selection import cross_val_score

# Realizar a validação cruzada com o modelo Random Forest
scores = cross_val_score(rf_model, X_train, y_train, cv=5)  # cv é o número de folds

# Calcular a média das pontuações
mean_accuracy = scores.mean()

print("Acurácia Média da Validação Cruzada:", mean_accuracy)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Acurácia Média da Validação Cruzada: 0.6142694565347602


In [49]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

# Definir as métricas que você deseja avaliar
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1': make_scorer(f1_score, average='weighted')
}

# Realizar a validação cruzada com o modelo Random Forest
cv_results = cross_validate(rf_model, X_train, y_train, cv=5, scoring=scoring)

# Imprimir os resultados de cada métrica em cada fold
print("Acurácia Média:", cv_results['test_accuracy'].mean())
print("Precisão Média:", cv_results['test_precision'].mean())
print("Recall Médio:", cv_results['test_recall'].mean())
print("F1-Score Médio:", cv_results['test_f1'].mean())

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` par

Acurácia Média: 0.6111048995727348
Precisão Média: 0.5984959270816452
Recall Médio: 0.6111048995727348
F1-Score Médio: 0.5918596532328737


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
